In [ ]:
#| include: false
from nbdev.showdoc import *

This part of the tutorial demonstrates how to use `NumerFrame` to download predictions bought on [NumerBay](http://numerbay.ai/) community marketplace. Currently only the main tournament is supported. Signals support will be added in future.

In [ ]:
from nbdev import show_doc
from numerblox.numerframe import create_numerframe, NumerFrame
from numerblox.download import NumeraiClassicDownloader
from numerblox.key import Key
from numerblox.model import NumerBayCSVs
from numerblox.submission import NumerBaySubmitter, NumeraiClassicSubmitter, NumeraiSignalsSubmitter

First, we download tournament data using `NumeraiClassicDownloader`.

In [ ]:
downloader = NumeraiClassicDownloader("numerframe_edu")
# Path variables
tournament_file = "numerai_tournament_data.parquet"
tournament_save_path = f"{str(downloader.dir)}/{tournament_file}"
# Download only tournament parquet file
downloader.download_single_dataset(tournament_file,
                                   dest_path=tournament_save_path)

📁 Downloading 'numerai_tournament_data.parquet' 📁

2022-04-08 08:30:58,816 INFO numerapi.utils: starting download
numerframe_edu/numerai_tournament_data.parquet: 582MB [00:59, 9.71MB/s]                              


Loading in data and initializing a `NumerFrame` takes one line of code. It will automatically recognize the data format such as `.csv` or `.parquet`. You have the option to add metadata, which is stored in the `meta` attribute.

In [ ]:
# Initialize NumerFrame from parquet file path
dataf = create_numerframe(tournament_save_path, metadata={"version": 3, "type": "tournament"})

In [ ]:
dataf.head(2)

,era,data_type,feature_dichasial_hammier_spawner,feature_rheumy_epistemic_prancer,feature_pert_performative_hormuz,feature_hillier_unpitied_theobromine,feature_perigean_bewitching_thruster,feature_renegade_undomestic_milord,feature_koranic_rude_corf,feature_demisable_expiring_millepede,...,target_paul_20,target_paul_60,target_george_20,target_george_60,target_william_20,target_william_60,target_arthur_20,target_arthur_60,target_thomas_20,target_thomas_60
id,,,,,,,,,,,,,,,,,,,,,
n000101811a8a843,0575,test,0.50,0.0,1.0,0.0,0.75,0.00,1.00,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
n001e1318d5072ac,0575,test,0.25,1.0,0.5,0.5,0.25,0.75,0.75,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Next, we create a NumerBayCSVs model object with a list of products we bought and NumerBay credentials. `numerbay_key_path` may be required for products that use client-side encryption. You can learn more about encryption [here](https://docs.numerbay.ai/updates/encryption)

In [ ]:
nb_model = NumerBayCSVs(data_directory='/app/notebooks/tmp',
                        numerbay_product_full_names=['numerai-predictions-numerbay'],  # change to the full names of products you bought
                        numerbay_username="your_username",  # change to your own username
                        numerbay_password="your_password",  # change to your own password
                        numerbay_key_path="/app/notebooks/tmp/numerbay.json")

Call the `predict` method on the `NumerFrame` to fetch the prediction file from NumerBay. If the file already exists in the `data_directory`, that file will be loaded without re-downloading.

In [ ]:
preds = nb_model.predict(dataf)

NumerBay submissions:   0%|          | 0/1 [00:00<?, ?it/s]

2022-04-08 08:32:39,576 INFO numerbay.utils: starting download

/app/notebooks/tmp/.numerai-predictions-numerbay.tmp:   0%|          | 0.00/30.2M [00:00<?, ?B/s]
/app/notebooks/tmp/.numerai-predictions-numerbay.tmp:   0%|          | 15.4k/30.2M [00:00<04:28, 113kB/s]
/app/notebooks/tmp/.numerai-predictions-numerbay.tmp:   0%|          | 48.1k/30.2M [00:00<03:39, 138kB/s]
/app/notebooks/tmp/.numerai-predictions-numerbay.tmp:   0%|          | 80.9k/30.2M [00:00<03:23, 148kB/s]
/app/notebooks/tmp/.numerai-predictions-numerbay.tmp:   0%|          | 146k/30.2M [00:00<01:54, 262kB/s] 
/app/notebooks/tmp/.numerai-predictions-numerbay.tmp:   1%|          | 212k/30.2M [00:00<01:26, 348kB/s]
/app/notebooks/tmp/.numerai-predictions-numerbay.tmp:   1%|          | 294k/30.2M [00:00<01:05, 454kB/s]
/app/notebooks/tmp/.numerai-predictions-numerbay.tmp:   2%|▏         | 458k/30.2M [00:01<00:39, 754kB/s]
/app/notebooks/tmp/.numerai-predictions-numerbay.tmp:   2%|▏         | 638k/30.2M [00:01<00:28, 1.0

The predictions are concatenated to the `NumerFrame` with column name `prediction_numerai-predictions-numerbay`

In [ ]:
preds

,era,data_type,feature_dichasial_hammier_spawner,feature_rheumy_epistemic_prancer,feature_pert_performative_hormuz,feature_hillier_unpitied_theobromine,feature_perigean_bewitching_thruster,feature_renegade_undomestic_milord,feature_koranic_rude_corf,feature_demisable_expiring_millepede,...,target_paul_60,target_george_20,target_george_60,target_william_20,target_william_60,target_arthur_20,target_arthur_60,target_thomas_20,target_thomas_60,prediction_numerai-predictions-numerbay
id,,,,,,,,,,,,,,,,,,,,,
n000101811a8a843,0575,test,0.50,0.00,1.00,0.00,0.75,0.00,1.00,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.50
n001e1318d5072ac,0575,test,0.25,1.00,0.50,0.50,0.25,0.75,0.75,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25
n002a9c5ab785cbb,0575,test,0.25,0.50,0.50,0.75,0.25,0.25,0.75,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25
n002ccf6d0e8c5ad,0575,test,0.50,1.00,0.50,1.00,0.50,1.00,0.75,0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.50
n0051ab821295c29,0575,test,0.50,0.00,0.00,0.25,0.00,1.00,0.50,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nffbdc800734996d,X,live,0.25,0.25,0.75,0.50,0.00,0.25,0.50,0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25
nffece39957d267b,X,live,0.50,0.75,0.75,0.75,0.50,0.25,0.50,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.50
nffedf2e8373db80,X,live,1.00,0.00,1.00,0.50,0.00,0.00,1.00,0.75,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00


In this part of the tutorial we have downloaded a prediction file from NumerBay with `NumerFrame`. This makes things easier for post processing such as ensembling and neutralization.

------------------------------------------------------

# NumerBay submission

This part of the tutorial is for sellers who want to upload their predictions to NumerBay to fulfill sale orders. Using `NumerBaySubmitter`, a seller can choose to submit to both Numerai and NumerBay or just NumerBay.

Assume we have some prediction column to upload for the Numerai main tournament, in this case the `prediction` column which simply takes the value of a feature.

In [ ]:
dataf = create_numerframe(tournament_save_path, metadata={"version": 3, "type": "tournament"})

In [ ]:
dataf['prediction'] = dataf['feature_dichasial_hammier_spawner']

Next, we create a `NumerBaySubmitter` object which wraps on top of a `NumeraiClassicSubmitter` object that you would normally use if you are only submitting to Numerai.

In [ ]:
key = Key(pub_id='YOUR_PUBLIC_KEY', secret_key='YOUR_PRIVATE_KEY')  # change to your own Numerai API key
numerai_submitter = NumeraiClassicSubmitter(directory_path="/app/notebooks/tmp", key=key)

🔑 Numerai Auth key initialized with pub_id = 'YOUR_PUBLIC_KEY' 🔑

Set `upload_to_numerai` to True (default) if you want to submit to both Numerai and NumerBay, set to False to submit only to NumerBay.

In [ ]:
nb_submitter = NumerBaySubmitter(tournament_submitter=numerai_submitter, upload_to_numerai=True, numerbay_username="numerbay", numerbay_password="your_password")

Finally, we call the `full_submission` method to perform the submission

In [ ]:
nb_submitter.full_submission(dataf, file_name='upload-full.csv', model_name='numerbay', numerbay_product_full_name='numerai-predictions-numerbay', cols='prediction')

📄 Saving predictions CSV to '/app/notebooks/tmp/upload-full.csv'. 📄

✈ NumerAPI: Uploading predictions from '/app/notebooks/tmp/upload-full.csv' for model 
'numerbay' (model_id='adabxxx-3acf-470e-8733-e4283261xxxx') ✈

2022-04-08 08:33:17,889 INFO numerapi.base_api: uploading predictions...


👍 NumerAPI submission of '/app/notebooks/tmp/upload-full.csv' for numerbay is successful! 👍

✈ NumerBay: Uploading predictions from '/app/notebooks/tmp/upload-full.csv' for NumerBay 
product 'numerai-predictions-numerbay' ✈

2022-04-08 08:34:00,032 INFO numerbay.numerbay: uploading artifact...
2022-04-08 08:34:00,576 INFO numerbay.numerbay: uploading encrypted artifact for order [1217] for [some_buyer]...
2022-04-08 08:34:01,069 INFO numerbay.utils: encrypting file


👍 NumerBay submission of '/app/notebooks/tmp/upload-full.csv' for NumerBay product 
numerai-predictions-numerbay is successful! 👍

The process for Signals submission is very similar and is omitted for brevity, just do the following:
- Use Signals NumerFrame
- Change `NumeraiClassicSubmitter` to `NumeraiSignalsSubmitter` for the `tournament_submitter` argument
- When calling `full_submission`, change the `cols` argument to the list of Signals column to submit (e.g. `['bloomberg_ticker', 'signal']`)

------------------------------------------------------

After we are done we can easily clean up our downloaded data with one line of code called from the downloader.

In [ ]:
# Clean up environment
downloader.remove_base_directory()

⚠ Deleting directory for 'NumeraiClassicDownloader' ⚠
Path: '/app/notebooks/numerframe_edu'